In [ ]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:1000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [ ]:
import ipywidgets as widgets
import pandas as pd
import io
import dropbox
import seaborn as sns
from IPython.display import display
from ipywidgets import interact_manual, interactive,GridspecLayout, AppLayout, Button, Layout



##get data from dropbox
DBX = dropbox.Dropbox('bb3H-rx_L8IAAAAAAAAPdR77ZBO4BYfjelCVEH8kkSj04MX4vx4RrPFfN0cfJ3tm')

_, res = DBX.files_download("/human_means.csv")

with io.BytesIO(res.content) as stream:
    human_means = pd.read_csv(stream, index_col=0)

_, res2 = DBX.files_download("/mouse_medians.csv")
with io.BytesIO(res2.content) as stream:
    meds = pd.read_csv(stream, index_col=0)   
    

# Get all gene and cell type names: HUMAN
human_listOfColumnNames = list(human_means.columns.values)
human_idx = list(human_means.index)
# Get all gene and cell type: MOUSE
mouse_listOfColumnNames = list(meds.columns.values)
idx = list(meds.index)




In [ ]:
##define GUI
table_style = {'description_width': 'initial'}
useidx = ('choose species','then genes will appear')
spacer = widgets.Label('')

In [ ]:
genes = []
genetitle = widgets.Label('Select Genes')
dropdown =  widgets.Combobox(
    options= idx,
    description='Gene',
    placeholder='Type here',
    layout={'width': '80%'})

def choose_gene(change):
    if len(genes)<=7:
        genes.append(dropdown.value)
    if len(genes)>7:       
        gene_button.button_style = 'warning'
        gene_button.description = "Max Genes Selected"
        gene_button.disabled = True
        
gene_button = widgets.Button(description = 'Enter Gene', button_style = 'success')
gene_button.on_click(choose_gene)

def choose_gene(change):
    if len(genes)<=7:
        genes.append(dropdown.value)
    if len(genes)>7:       
        gene_button.button_style = 'warning'
        gene_button.description = "Max Genes Selected"
        gene_button.disabled = True
        
gene_button = widgets.Button(description = 'Enter Gene', button_style = 'success')
gene_button.on_click(choose_gene)

In [ ]:

analysislabel = widgets.Label('Output Type:')
expgraph = widgets.Button(
    description='Expression Values',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    width = '110%',
    display='flex',
    flex_flow='column',
    align_items='stretch')

venn = widgets.Button(
    description='Venn Diagram',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    width = '110%',
    display='flex',
    flex_flow='column',
    align_items='stretch')

def on_exp_button_clicked(expgraph):
    graphtype = "Exp"
    venn.disabled = True
    expgraph.button_style = 'success'
expgraph.on_click(on_exp_button_clicked)

def on_venn_button_clicked(expgraph):
    graphtype = "Venn"
    venn.button_style = 'success'
    expgraph.disabled = True
venn.on_click(on_venn_button_clicked)

In [ ]:

choicelabel = widgets.Label('Analyze species:')
msCells = widgets.Button(
    description='Mouse',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    width = '110%')

huCells = widgets.Button(
    description='Human',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    width = '110%')

#####################################################

mscelllabel = widgets.Label('Mouse Cell Clusters:')
msCells_drop =widgets.SelectMultiple(
    options=mouse_listOfColumnNames,
    disabled=True,
    rows=17,
    layout={'width': 'max-content'},
    #style=table_style
) 

hucelllabel = widgets.Label('Human Cell Clusters:')
huCells_drop =widgets.SelectMultiple(
    options=human_listOfColumnNames,
    disabled=True,
    rows=17,
    layout={'width': 'max-content'},
    #style=table_style
) 

def choose_mouse(msCells):
    species = "Mouse"
    huCells.disabled = True
    msCells.button_style = 'success'
    msCells_drop.disabled = False
    huCells_drop.disabled = True
msCells.on_click(choose_mouse)

def choose_human(huCells):
    species = "Human"
    huCells.button_style = 'success'
    msCells.disabled = True
    msCells_drop.disabled = True
    huCells_drop.disabled = False
huCells.on_click(choose_human)

########################################
go_button = widgets.Button(description = 'Produce Output', 
                           button_style = 'success', 
                            display='flex',
                            flex_flow='column',
                            align_items='stretch')
reset_button = widgets.Button(description = 'Reset Options', button_style = 'warning',width = 'auto')

cells =[]
def on_go_button_clicked(go_button):
    cells.append(msCells_drop.value)

go_button.on_click(on_go_button_clicked)



submitms_button = widgets.Button(description = 'Submit Mouse Cells', button_style = 'success',width = 'auto')
submithu_button = widgets.Button(description = 'Submit Human Cells', button_style = 'success',width = 'auto')

spacervenn = widgets.VBox([spacer,venn])
specieschoices = widgets.VBox([choicelabel,msCells, mscelllabel, msCells_drop,submitms_button])
spacerhuman = widgets.VBox([spacer, huCells,hucelllabel, huCells_drop,submithu_button])
genesbox = widgets.VBox([genetitle,dropdown, gene_button])
buttons = widgets.HBox([go_button, spacer, reset_button])
graphchoices = widgets.VBox([analysislabel, expgraph, venn, spacer, spacer, spacer, genesbox, spacer, spacer, buttons])
graphs2 = widgets.HBox([graphchoices])
####################################################

In [ ]:

#def create_expanded_button(description, button_style):
    #return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))

box_layout = widgets.Layout(
        #border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px',
        height='auto', width='auto')

grid = GridspecLayout(3, 4)
grid.layout = box_layout
# fill it in with widgets
grid[0,0] = specieschoices
grid[0,1] = spacerhuman
#grid[,] = genesbox
grid[0, 2:] = graphs2
#grid[0,3] = spacervenn

#grid[2, 2] = go_button
#grid[2,3]= reset_button
grid

In [ ]:
print(genes)
cells = [item for sublist in cells for item in sublist]
print(cells)

In [ ]:

newdf = pd.DataFrame(columns = ['Cells','Genes','Expression'])
#cells = [item for sublist in cells for item in sublist]
newdf['Cells'] = cells * len(genes)


genelist=[]
for i in range(len(genes)):
    genea = [genes[i]]*len(cells)
    genelist.append(genea)
flat_geneslist = [item for sublist in genelist for item in sublist]
newdf['Genes'] = flat_geneslist

expressions =[]
for i in range(len(genes)*len(cells)):
    value1 = meds.loc[newdf['Genes'][i], newdf['Cells'][i]]
    expressions.append(value1)
newdf['Expression'] = expressions

maxm = max(newdf["Expression"])

In [ ]:
display(newdf, maxm)

In [ ]:
###Now graph it####################################################
    
sns.set_palette("Blues")
g = sns.catplot(x="Genes", y="Expression", col="Cells",
                data=newdf, saturation=1,
                kind="bar", ci=None, aspect=.6, edgecolor="k", linewidth=1)
(g.set_axis_labels("", "Expression")
  .set_xticklabels(genes)
  .set_titles("{col_name}")
  .set(ylim=(0, (maxm + 1)))
  .despine(left=True))  

for ax in g.axes.ravel():
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
    for p in ax.patches:
        ax.annotate('%.2f' % p.get_height(), (p.get_x() + p.get_width() / 2., p.get_height()), 

                    ha='center', va='center', xytext=(0, 8), 
                    textcoords='offset points')

##Create an excel file ##################
if Excel == True:
    FileName = app.getEntry("Excel Name")
    FileName = FileName + ".xlsx"
    newdf.to_excel(FileName) 